#   Лабораторная работа 4. Анализ и визуализация данных. Вариант 3


In [10]:
import pandas as pd
import csv
import typing
from typing import List, Tuple
import random
import matplotlib.pyplot as plt
import cv2

def read_file(path: str) -> List[Tuple[str, str]]:
    '''Читает файл path и возвращает список кортежей с именами файлов и их классами'''
    files: List[Tuple[str, str]] = []
    with open(path, "r") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=",")
        for row in reader:
            files.append((row["full_path"], row["class"]))
    return files

list_information: List[Tuple[str, str]] = read_file("annotation.csv")


1. Создание Dataframe с двумя колонками (название класса и абсолютный путь к файлу)

In [8]:
"""
 Создание датафрейма

    Parameters
    ----------
    list_information
     Список файлов из аннотации
    Returns
    -------
    pd.DataFrame
     Датафрейм в 2 колонки
 """
def create_df(list_information)->pd.DataFrame:
    df = pd.DataFrame(columns=["Class", "Abs_Path"])
    for file_info in list_information:
        df.loc[len(df)] = [file_info[1], file_info[0]]
    return df

create_df(list_information)

,Class,Abs_Path
0,cats,/Users/polina/Documents/Programm-lab/dataset/c...
1,cats,/Users/polina/Documents/Programm-lab/dataset/c...
2,cats,/Users/polina/Documents/Programm-lab/dataset/c...
3,cats,/Users/polina/Documents/Programm-lab/dataset/c...
4,cats,/Users/polina/Documents/Programm-lab/dataset/c...
...,...,...
1995,dogs,/Users/polina/Documents/Programm-lab/dataset/d...
1996,dogs,/Users/polina/Documents/Programm-lab/dataset/d...
1997,dogs,/Users/polina/Documents/Programm-lab/dataset/d...
1998,dogs,/Users/polina/Documents/Programm-lab/dataset/d...


2-4. Именование колонок, числовая метка, информация о высоте, ширине и глубине  изображения

In [11]:
"""
 Добавление нескольких колонок по метке

    Parameters
    ----------
    list_information
     Список файлов из аннотации
    Returns
    -------
    pd.DataFrame
     Датафрейм 
 """
def create_df(list_information)->pd.DataFrame:
    df = pd.DataFrame(columns=["Class", "Abs_Path", "Id", "Weight", "Height", "Depth"])
    for file_info in list_information:
        id = (0 if "cats" == file_info[1] else 1)
        im = cv2.imread(file_info[0])
        H,W,D = im.shape
        df.loc[len(df)] = [file_info[1], file_info[0], id, W,H,D]
    return df

df = create_df(list_information)
df

,Class,Abs_Path,Id,Weight,Height,Depth
0,cats,/Users/polina/Documents/Programm-lab/dataset/c...,0,448,320,3
1,cats,/Users/polina/Documents/Programm-lab/dataset/c...,0,480,300,3
2,cats,/Users/polina/Documents/Programm-lab/dataset/c...,0,480,320,3
3,cats,/Users/polina/Documents/Programm-lab/dataset/c...,0,448,320,3
4,cats,/Users/polina/Documents/Programm-lab/dataset/c...,0,480,270,3
...,...,...,...,...,...,...
1995,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,300,3
1996,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,248,320,3
1997,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,319,3
1998,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,289,3


5. Вычисление статистической информации


In [12]:
"""
 Вычисление информации о размере изображений

    Parameters
    ----------
    df
     Датафрейм
    Returns
    -------
    pd.DataFrame
     Датафрейм со значениями
 """
def get_static_info(df)->pd.DataFrame:
    
    return {"Height": df["Height"].mean(), "Weight": df["Weight"].mean(),
            "Depht":  df["Depth"].mean(), "Id": df["Id"].mean()}

get_static_info(df)

{'Height': 305.3425, 'Weight': 444.026, 'Depht': 3.0, 'Id': 0.5}

6. Отфильтрованный по метке класса DataFrame

In [14]:
"""
 Фильтрация датафрейма

    Parameters
    ----------
    df
     Датафрейм
    id
     Идентификатор класса

    Returns
    -------
    pd.DataFrame
     Датафрейм
 """
def filter_class_id(df, id) -> pd.DataFrame:
    return df.loc[ df["Id"] == id ]

filter_class_id(df, 1)

,Class,Abs_Path,Id,Weight,Height,Depth
1000,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,320,3
1001,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,320,3
1002,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,400,320,3
1003,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,300,3
1004,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,320,3
...,...,...,...,...,...,...
1995,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,300,3
1996,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,248,320,3
1997,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,319,3
1998,dogs,/Users/polina/Documents/Programm-lab/dataset/d...,1,480,289,3


7. Отфильтрованный по заданным параметрам DataFrame

In [15]:
"""
 Фильтрация датафрейма по заданным параметрам

    Parameters
    ----------
    df
     Датафрейм
    weight
     Ширина
    height
     Высота
    id
     Идентификатор класса

    Returns
    -------
    pd.DataFrame
     Датафрейм
 """
def filter_given_parameters(df, weight, height, id)->pd.DataFrame:
     return df.loc[ (df["Id"] == id) & (df["Weight"] <= weight) & (df["Height"] <= height) ]

filter_given_parameters(df, 320, 320, 1)

,Class,Abs_Path,Id,Weight,Height,Depth
